# 0.0 LIBRARY

In [1]:
#!pip install umap-learn -q

In [2]:
#!python3 -c *import import umap.umap_ as umap:print('umap SDK version:{}'.format{umap._version_})

In [3]:
#!pip freeze # umap 0.5.3

In [4]:
import pandas     as pd
import numpy      as np
import umap.umap_ as umap
from typing       import Tuple
from functools    import reduce
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold      import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.ensemble      import RandomForestRegressor
from sklearn.metrics       import silhouette_score
from google.cloud          import bigquery

# 1.0 Loading data

In [5]:
%%bigquery df_raw

SELECT * FROM `braided-period-442813-v0.ecommerce_cluster_insider.dados_ecommerce_iterm` LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,string_field_8
0,560604,23209,LUNCH BAG DOILEY PATTERN,100,2017-07-18,1.45,15251.0,United Kingdom,None
1,560604,22993,SET OF 4 PANTRY JELLY MOULDS,72,2017-07-18,1.06,15251.0,United Kingdom,None
2,560604,22848,BREAD BIN DINER STYLE PINK,4,2017-07-18,14.95,15251.0,United Kingdom,None
3,560604,22699,ROSES REGENCY TEACUP AND SAUCER,24,2017-07-18,2.55,15251.0,United Kingdom,None
4,560604,22457,NATURAL SLATE HEART CHALKBOARD,48,2017-07-18,2.55,15251.0,United Kingdom,None


In [7]:
df_raw.shape

(1000, 9)

In [8]:
df_raw.drop(labels = 'string_field_8', axis = 1, inplace = True)

# 2.0 Data preparation

In [9]:
df_raw.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         Int64
InvoiceDate     dbdate
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [10]:
# Trocas os tipos 
#df_raw['CustomerID'] = df_raw['CustomerID'].astype(int)

In [11]:
#df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])

In [12]:
def coluna_para_int( dataframe: pd.DataFrame, nome_coluna: str) -> bool:
    """
    Descrição:
        Converte a coluna especificada em um datafreme parao tipo int
    
    Args: 
         dataframe(pd.DataFrame): O dataframe a ser processado.
         column_name (str): O nome da coluna a ser convertida.
    Returns:
         bool: True se a conversão foi bem sucedida. False caso contrário.
    """
    if nome_coluna not in dataframe.columns:
        raise ValueError(f'Coluna {nome_coluna} não encontrado')
    try:    
        dataframe[nome_coluna] = dataframe[nome_coluna].astype(int)
    except:
        raise ValueError(f'Não foi possível converter a coluna {nome_coluna} para inteiro.')
    return True

In [13]:
def coluna_para_date( dataframe: pd.DataFrame, nome_coluna: str, formato_data: str = None) -> bool:
    """
    Descrição:
        Converte a coluna especificada em um datafreme parao tipo data
    
    Args: 
         dataframe(pd.DataFrame): O dataframe a ser processado.
         column_name (str): O nome da coluna a ser convertida.
    Returns:
         bool: True se a conversão foi bem sucedida. False caso contrário.
    """
    if nome_coluna not in dataframe.columns:
        raise ValueError(f'Coluna {nome_coluna} não encontrado no dataframe de entrada')
    
    try:  
        if formato_data:
            dataframe[nome_coluna] = pd.to_datetime(dataframe[nome_coluna], format = formato_data)
        else:
            dataframe[nome_coluna] = pd.to_datetime(dataframe[nome_coluna])
    except (ValueError, TypeError):
        raise ValueError(f'Could not convert column {nome_coluna}')
    # Retorna True se a conversão for bem sucedida
    return True

In [14]:
coluna_para_int( df_raw, 'CustomerID')

True

In [15]:
# Dois dataframes, compres e retornos


def keep_features(dataframe: pd.DataFrame, keep_columns: list) -> pd.DataFrame:
    """
    Retorna um DataFrame com as colunas especificadas em keep_colunas
    
    Args:
        dataframe (pd.DataFrame): O DataFrame a ser processado.
        keep_columns (list): A lista de nomes de colunas a serem mantidas no DataFrame resultante. 
    Returns:
        pd.DataFrame: O DataFrame resultante com apenas as colunas especificadas em keep_columns.
    """
    return dataframe[keep_columns]

def filtragem_features(dataframe_raw: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Descrição
    Args:
    Retornos: 
    """ 
    df_returns   = dataframe_raw.loc[dataframe_raw['Quantity']<0, ['CustomerID', 'Quantity']]
    
    df_purchases = dataframe_raw.loc[dataframe_raw['Quantity'] >= 0, :]
    
    # Filter main data
    df_filtered  = keep_features( dataframe_raw, ['InvoiceNo' , 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice', 
                                                  'CustomerID', 'Country'])
    return df_filtered, df_purchases, df_returns
    

In [16]:
df_filtered, df_purchases, df_returns = filtragem_features(df_raw)

In [17]:
df_filtered.head(1)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,560604,23209,100,2017-07-18,1.45,15251,United Kingdom


In [18]:
df_purchases.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,560604,23209,LUNCH BAG DOILEY PATTERN,100,2017-07-18,1.45,15251,United Kingdom


In [19]:
df_returns.head()

,CustomerID,Quantity
24,17581,-2
25,17581,-4
26,17581,-8
27,16255,-2
28,16255,-1


# 3.0 Feature engineering


In [20]:
def calculate_gross_revenue(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a receita bruta de cada cliente com base nas colunas 'Quantity' e 
    'UnitPrice' e retorna um DataFrame com as colunas 'CustomerID' e 'gross_revenue'.
    Args: 
        dataframe_purchases (pd.Dataframe): O DataFrame das compras contendo as colunas
        'CustomerID', 'Quantity'
    Returns:
        pd.DataFrame: O DataFrame resultante contendo as colunas 'CustomerID' e 'grass revenue',
    """
    # Verifica se as colunas necessárias estão presentes ou DataFrame de entrada
    required_columns = {'CustomerID', 'Quantity', 'UnitPrice'}
    missing_columns  = required_columns - set(dataframe_purchases.columns)
    if missing_columns: 
        raise ValueError(f"O DataFrame de entrada está faltando as sequintes colunas: {missing_columns}")
        
        # Calcula a receita bruta e agrupa por CustomerID
    df = dataframe_purchases.copy()
    df.loc[:, 'gross_revenue'] = df.loc[:, 'Quantity'] * df.loc[:, 'UnitPrice']
    grouped_df = df.groupby('CustomerID').agg({'gross_revenue':'sum'}).reset_index().copy()
    
    return grouped_df

def create_recency(dataframe_purchases: pd.DataFrame, dataframe_filtered: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula a recência da última compra para cada cliente.
    
    Args:
        dataframe_purchases(pd.DataFrame): DataFrame com as informações de compras de todos os clientes.
        dataframe_filtered (pd.DataFrame): DataFrame filtrado apenas com as informaões dos clientes que des
        
    Returns: 
        pd.DataFrame: DataFrame com as colunas 'CustomerID' e 'recency_days', indicando a recẽncia em dias de
        
    """
    required_columns = {'CustomerID', 'InvoiceDate'}
    missing_columns  = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    # calcula a data da útima compra de cada cliente.
    df_recency = dataframe_purchases.loc[:,['CustomerID', 'InvoiceDate']].groupby('CustomerID').max().reset_index()
    
    # calcula a recência em dias da última compra de cada cliente em relação à data mais recente da base de
    dataframe_filtered['InvoiceDate'] = pd.to_datetime( dataframe_filtered['InvoiceDate'])
    df_recency['InvoiceDate']         = pd.to_datetime( df_recency['InvoiceDate'])
    df_recency['recency_days'] = (dataframe_filtered['InvoiceDate'].max() - df_recency['InvoiceDate']).dt.days
    
    # retorna o DataFrame apenas com as colunas 'CustomerID' e 'recency_days'
    return df_recency[['CustomerID', 'recency_days']]

def create_quantity_purchased( dataframe_purchases: pd.DataFrame ) -> pd.DataFrame:
    """
    Calcula a quantidade de produtos adquiridos por cada cliente.
    
    Args: 
        dataframe_purchases (pd.DataFrame): DataFrame com as informações de compras de todos os clientes.
        
    Returns:
        pd.DataFrame: DataFrame com as colunas 'CustomerID' e 'qty_products', indicando a quantidade de produtos
        
    """
    required_columns = {'CustomerID', 'StockCode'}
    missing_columns  = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    #agrupa as informações de compras por 'CustomerID' e conta o número de StockCode para cada grupo
    qty_purchased = dataframe_purchases.loc[:, ['CustomerID', 'StockCode']].groupby('CustomerID').count()
                         
    # renomeia o DataFrame com as colunas 'CustomerID' e 'qty_products'
    qty_purchased = qty_purchased.reset_index().rename( columns={'StockCode': 'qty_products'})
    #retorna a DataFrame com as coclunas 'CustomerID' e 'qty_products'
    return qty_purchased
    
def create_freq_purchases(dataframe_purchases: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the purchase frequency of each customer based on the purchase history. 
    
    Parameters
    
    ----------
    pd.DataFrame
        DataFrame with the purchase frequency of each customer, containing columns CustomerID and frequency
    """
    required_columns = { 'CustomerID', 'InvoiceNo', 'InvoiceDate'}
    missing_columns  = required_columns - set(dataframe_purchases.columns)
    if missing_columns:
        raise ValueError(f"O DataFrame de entrada está faltando as seguintes colunas: {missing_columns}")
    # Calculate time range of purchases for each customer
    df_aux = (dataframe_purchases[['CustomerID', 'InvoiceNo', 'InvoiceDate']]
              .drop_duplicates()
              .groupby('CustomerID')
              .agg(max_ = ('InvoiceDate', 'max'), 
                   min_ = ('InvoiceDate', 'min'),
                   days_= ('InvoiceDate', lambda x: (( x.max() - x.min()).days) + 1),
                   buy_ = ('InvoiceNo', 'count')).reset_index())
                         
    # Calculate frequency of purchases for each customer
    df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(
        lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1)
    
    return df_aux

def create_qty_returns(dataframe_returns:pd.DataFrame) -> pd.DataFrame:
    """
    Computes the total quantity of return products for each customer
    
    Args:
        dataframe_returns: A pandas DataFrame cocntaining information about returns.
    Returns: 
        A pandas DataFrame with the total quantity of returned product for each customer.
    """
    # Validate input data
    if not all( col in dataframe_returns.columns for col in [ 'CustomerID', 'Quantity']):
            raise ValueError("Input DataFrame must contain 'CustomerID' and 'Quantity' columns")
    
    # Compupte quantity of rerturns
    df_returns = dataframe_returns[['CustomerID', 'Quantity']].groupby('CustomerID').sum().reset_index().rename(columns = {'Quantity':'qty_returns'})
    df_returns['qty_returns'] = df_returns['qty_returns']* -1
    
    return df_returns

def run_feature_engineering( dataframe_filtered: pd.DataFrame, dataframe_purchases: pd.DataFrame, dataframe_returns: pd.DataFrame) -> pd.DataFrame:
    """
    Performs feature engineering on the input dataframes and returns a new dataframe with the engineered features
    
    Args: 
        dataframe_filtered : A pandas DataFrame containing filtered customer order data.
        dataframe_purchases: A pandas DataFrame containing customer purchase data. 
        dataframe_returns  : A pandas DataFrame containing customer return data.
        
    Returns:
        A pandas DataFrame with the engineered features for each customer.
    """
    # Check if input dataframes are empty
    if dataframe_filtered.empty:
        raise ValueError("Input DataFrame 'dataframe_filtered' is empty")
    if dataframe_purchases.empty:
        raise ValueError("Input DataFrame 'dataframe_purchases' is empty")
 
    # Check if required_columns are present in input daframes
    required_columns = ['CustomerID', 'InvoiceDate', 'StockCode', 'Quantity', 'UnitPrice']
    for df, name in zip( [dataframe_filtered, dataframe_purchases], ['dataframe_iltererd', 'dataframe_purchases']):
        missing_columns = set( required_columns) - set(df.columns)
        if missing_columns:
            raise ValueError(f"Missing_columns {missing_columns} in input DataFrame '{name}'")
        if 'CustomerID' not in dataframe_returns.columns:
            raise ValueError( "Column 'CustomerID' not found in input DataFrame 'dataframe_returns'")
        if 'Quantity' not in dataframe_returns.columns: 
                         raise ValueError("Column 'Quantity' not found in input DataFrame 'dataframe_returns'")
                         
        # Perform feature engineering 
        df_fengi        = keep_features(dataframe_filtered,['CustomerID']).drop_duplicates( ignore_index = True)
        gross_revenue   = calculate_gross_revenue(dataframe_purchases)
        df_recency      = create_recency( dataframe_purchases, dataframe_filtered)
        df_qty_products = create_quantity_purchased( dataframe_purchases)
        df_freq         = create_freq_purchases( dataframe_purchases)
        returns         = create_qty_returns( dataframe_returns)
        
        # Merge dataframes
        dfs             = [df_fengi, gross_revenue, df_recency, df_qty_products, df_freq, returns]
        df_fengi        = reduce( lambda left,right: pd.merge( left, right, on = 'CustomerID', how = 'left'), dfs)
        
        # Fil NaN values
        df_fengi['qty_returns'] = df_fengi['qty_returns' ].fillna(0)
         
        # Select final features and return dataframe
        features        = ['CustomerID', 'gross_revenue','recency_days', 'qty_products', 'frequency', 'qty_returns'] 
        
        return keep_features( df_fengi, features).dropna()

In [21]:
df_fengi = run_feature_engineering(df_filtered, df_purchases, df_returns)
df_fengi.head()

/var/tmp/ipykernel_3380/1967922874.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_filtered['InvoiceDate'] = pd.to_datetime( dataframe_filtered['InvoiceDate'])


,CustomerID,gross_revenue,recency_days,qty_products,frequency,qty_returns
0,15251,464.72,0.0,5.0,1.0,0
1,17306,611.24,0.0,4.0,1.0,0
2,12510,217.52,0.0,3.0,1.0,0
3,13854,313.76,0.0,4.0,1.0,0
4,15587,111.96,0.0,2.0,1.0,0


# 4.0 EDA 


In [22]:
df_eda = keep_features( df_fengi, ['gross_revenue', 'recency_days', 'qty_products', 'frequency', 'qty_returns'])
df_eda.head()

,gross_revenue,recency_days,qty_products,frequency,qty_returns
0,464.72,0.0,5.0,1.0,0
1,611.24,0.0,4.0,1.0,0
2,217.52,0.0,3.0,1.0,0
3,313.76,0.0,4.0,1.0,0
4,111.96,0.0,2.0,1.0,0


# 5.0 Experimentation

## 5.1. K clusters

In [23]:
k_clusters = np.arange(2,5,1)
k_clusters

array([2, 3, 4])

## 5.2 Scaler

In [24]:
scaler = MinMaxScaler()

## 5.3 Algoritmos de redução de dimensionalidade

In [25]:
def train_pca( 
    scaler, dataframe: pd.DataFrame, to_scaling: bool = True, **kwargs
) -> pd.DataFrame:
    """
    Treina o modelo PCA no dataframe de entrarda e aplica o aos dados usando o scaler especificado
    
    Args:  
        scaler: objeto scaler para transformar os dados 
        dataframe  ( pd.DataFrame ): detaframe de entrada
        k_clusters ( int ): número de clusters para os algoritimos de clustering
        to_scaling ( bool ): indica se os dados devem ser escalados antes de cplica o PCA (padrão True)
    **kwargs: argumentos adicionais para a inicialização do modelo PCA
    
    Raises:
        ValueError: Se o dataframe de entrada estiver vazio
    """
    if dataframe.empty:
        raise ValueError('Input dataframe is empty.')
    if to_scaling:
        X = scaler.fit_transform( dataframe)
    else:
        X = dataframe.copy()
    pca = PCA(**kwargs)
    principal_components = pca.fit_transform(X)
    df_pca = pd.DataFrame(principal_components)
    return df_pca

def train_umap( 
    scaler, dataframe: pd.DataFrame, to_scaling: bool = True, **kwargs
)-> pd.DataFrame:
    """
    
    """
    if dataframe.empty:
        raise ValueError('Input dataframe is empty.')
    if to_scaling:
        X = scaler.fit_transform(dataframe)
    else:
        x = dataframe.copy
        
    reducer = umap.UMAP(random_state = 42, **kwargs)
    embedding = reducer.fit_transform(X)
    
    df_umap = pd.DataFrame()
    df_umap['embedding_x'] = embedding[:, 0]
    df_umap['embedding_y'] = embedding[:, 1]
    return df_umap

def train_tsne(
   scaler, dataframe: pd.DataFrame, to_scaling: bool = True, **kwargs
)-> pd.DataFrame:
    """
    """
    if dataframe.empty:
        raise ValueError("Input dataframe is empty.")
    if to_scaling:
        X = scaler.fit_transform(dataframe)
    else:
        X = dataframe.copy()
    
    reducer = TSNE(random_state = 42, **kwargs)
    embedding = reducer.fit_transform(X)
    
    df_tsne = pd.DataFrame()
    df_tsne['embedding_x'] = embedding[:, 0]
    df_tsne['embedding_y'] = embedding[:, 1]
    return df_tsne

def train_tree_embedding( scaler, dataframe:pd.DataFrame, 
                         target: str = 'gross_revenue', **kwargs
                        ) -> pd.DataFrame:
    """
    """
    #Separate features and target varible
    X = dataframe.drop( columns = [target], axis = 1 )
    X = pd.DataFrame(scaler.fit_transform(X.values), columns = X.columns,
                     index = X.index)
    y = dataframe[target]
    
    # Define and train the model
    rf_model = RandomForestRegressor(random_state = 42, **kwargs)
    rf_model.fit(X,y)
    
    # Compute leaf indices for each simple 
    leaf_indices = rf_model.apply(X)
    
    # Create DataFrame with leaf_indices
    df_leaf = pd.DataFrame( leaf_indices)
    
    return df_leaf

In [26]:
df_pca = train_pca(scaler, df_eda, n_components = 2 )

In [27]:
df_umap = train_umap(scaler, df_eda)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [28]:
df_tsne = train_tsne( scaler, df_eda )

In [29]:
df_tree = train_tree_embedding(scaler, df_eda )

In [30]:
df_pca.head()

,0,1
0,-0.129637,-0.166202
1,-0.110078,-0.191893
2,-0.177194,-0.157525
3,-0.157805,-0.158821
4,-0.198078,-0.155192


In [31]:
df_umap.head()

,embedding_x,embedding_y
0,0.640449,5.697544
1,1.041892,5.568538
2,0.568434,6.483901
3,0.544053,6.181122
4,0.164315,6.482210


In [32]:
df_tree.head(2)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,11,11,13,12,7,7,6,8,6,5,...,8,6,11,7,10,5,9,7,9,12
1,10,11,12,11,6,6,5,8,6,4,...,8,5,10,6,9,5,8,6,8,11


## 5.4 Clustering Algorithms

In [33]:
def train_kmeans(dataframe: pd.DataFrame, k_clusters: list, **kwargs) -> list:
    """
    treine modela KMeans para diferentes valores de k_clusters e calcula a pontuação
    da silhueta para cada um.
    
    Args: 
        dataframe: um dataframe com as features de treinamento 
        k_clusters: uma lista de inteiros que representam o número de clusters a serem 
        testados.
        **kwargs: argumentos adicionais para a inicialização do modelo de mistura gaussiana
        
    Returns: 
        uma lista de pontuações de silhueta correspondentes a cada valor de k_clusters.
    """
    
    silhouette_scores = []
    
    for k in k_clusters: 
        # Model training
        kmeans_model = KMeans(n_clusters = k, random_state = 42, **kwargs).fit(dataframe)
        
        # Model predict
        labels = kmeans_model.predict(dataframe)
        
        # Model performance
        silhouette = silhouette_score(dataframe, labels, metric = 'euclidean')
        silhouette_scores.append(silhouette)
     
    return silhouette_scores

def train_gmm( dataframe: pd.DataFrame, k_clusters: list, **kwargs) -> list:
    """
    """
    silhouette_scores = []
    
    for k in k_clusters: 
        # model training
        gmm_model = GaussianMixture(n_components = k, random_state = 42, **kwargs).fit(dataframe)
        
        labels = gmm_model.predict(dataframe)
        
        # Model Performance
        silhouette = silhouette_score(dataframe, labels, metric = 'euclidean')
        silhouette_scores.append(silhouette)
    
    return silhouette_scores

In [34]:
kmeans_list =  train_kmeans( df_pca, k_clusters, n_init = 10)

In [35]:
gmm_list =  train_gmm( df_pca, k_clusters, n_init = 10)

In [36]:
k_clusters

array([2, 3, 4])

In [37]:
kmeans_list

[0.6220741584917507, 0.5372544896651817, 0.6274491596268614]

In [38]:
gmm_list

[0.626615506534057, 0.627400195924751, 0.12336904282013061]

In [39]:
df_results_pca = pd.DataFrame({
                                'scaler': str(scaler),
                                'Reducer': 'PCA',
                                'Cluster': k_clusters, 
                                'KMeans': kmeans_list,
                                'GCM': gmm_list
                                })

In [40]:
df_results_pca

,scaler,Reducer,Cluster,KMeans,GCM
0,MinMaxScaler(),PCA,2,0.622074,0.626616
1,MinMaxScaler(),PCA,3,0.537254,0.627400
2,MinMaxScaler(),PCA,4,0.627449,0.123369


In [41]:
gmm_list    = train_gmm(df_umap, k_clusters, n_init = 10)
kmeans_list = train_kmeans( df_umap, k_clusters, n_init = 10)

df_results_umap = pd.DataFrame({
                                'scaler' : str(scaler),
                                'Reducer': 'UMAP',
                                'Cluster': k_clusters, 
                                'KMeans' : kmeans_list,
                                'GCM'    : gmm_list
                                })
df_results_umap

,scaler,Reducer,Cluster,KMeans,GCM
0,MinMaxScaler(),UMAP,2,0.592400,0.592400
1,MinMaxScaler(),UMAP,3,0.464587,0.407229
2,MinMaxScaler(),UMAP,4,0.497216,0.302360


In [42]:
df_leaf               = train_tree_embedding( scaler, df_eda, n_jobs = -1, n_estimators = 10)

df_tree_tsne          = train_tsne( scaler, df_leaf, to_scaling = False )
gmm_list_tree_tsne    = train_gmm( df_tree_tsne, k_clusters, n_init = 10)
kmeans_list_tree_tsne = train_kmeans( df_tree_tsne, k_clusters, n_init = 10)
df_results_tree_tsne  = pd.DataFrame({
                                'scaler' : str(scaler),
                                'Reducer': 'Tree_embedding_tsne',
                                'Cluster': k_clusters, 
                                'KMeans' : kmeans_list_tree_tsne,
                                'GCM'    : gmm_list_tree_tsne
                                })
df_results_tree_tsne

,scaler,Reducer,Cluster,KMeans,GCM
0,MinMaxScaler(),Tree_embedding_tsne,2,0.616358,0.602082
1,MinMaxScaler(),Tree_embedding_tsne,3,0.547061,0.526369
2,MinMaxScaler(),Tree_embedding_tsne,4,0.494842,0.413007


In [43]:
df_tree_pca         = train_pca(scaler, df_leaf, to_scaling = False)


gmm_list_tree_pca   = train_gmm( df_tree_pca, k_clusters, n_init = 100)
kmeans_list_tree_pca = train_kmeans( df_tree_pca, k_clusters, n_init = 10)

df_results_tree_pca = pd.DataFrame({
                                'scaler' : str(scaler),
                                'Reducer': 'Tree_embedding_pca',
                                'Cluster': k_clusters, 
                                'KMeans' : kmeans_list_tree_pca,
                                'GCM'    : gmm_list_tree_pca, 
                                })
df_results_tree_pca

,scaler,Reducer,Cluster,KMeans,GCM
0,MinMaxScaler(),Tree_embedding_pca,2,0.558770,0.558770
1,MinMaxScaler(),Tree_embedding_pca,3,0.507156,0.493844
2,MinMaxScaler(),Tree_embedding_pca,4,0.468778,0.468778


In [46]:
df_final_results = pd.concat([df_results_pca, df_results_umap, df_results_tree_tsne, df_results_tree_pca], axis = 0)
df_final_results

,scaler,Reducer,Cluster,KMeans,GCM
0,MinMaxScaler(),PCA,2,0.622074,0.626616
1,MinMaxScaler(),PCA,3,0.537254,0.627400
2,MinMaxScaler(),PCA,4,0.627449,0.123369
0,MinMaxScaler(),UMAP,2,0.592400,0.592400
1,MinMaxScaler(),UMAP,3,0.464587,0.407229
2,MinMaxScaler(),UMAP,4,0.497216,0.302360
0,MinMaxScaler(),Tree_embedding_tsne,2,0.616358,0.602082
1,MinMaxScaler(),Tree_embedding_tsne,3,0.547061,0.526369
2,MinMaxScaler(),Tree_embedding_tsne,4,0.494842,0.413007
0,MinMaxScaler(),Tree_embedding_pca,2,0.558770,0.558770


## 5.6 Save Results

In [48]:
# Salvar no bigquery
def salvar_no_bigquery(
    dataframe         : pd.DataFrame, 
    project_name      : str, 
    dataset_table_name: str ): 
    
    client = bigquery.Client(project = project_name)
    
    job = client.load_table_from_dataframe(dataframe, dataset_table_name)
    job.result() # tempo 58 do vídeo aula 6

In [49]:
salvar_no_bigquery(df_final_results,
                   project_name = 'braided-period-442813-v0',
                  dataset_table_name = 'ecommerce_cluster_insider.resultado_experimentos_clustering')
# no bigquery minha pasta de projecto se chama braided-period-442813-v0, nessa minha pasta de projectos temos uma subpasta 
# ou schema de tabelas chamada ecommerce_cluster_insider
# nesse estamos criando uma tabela resutado_exeperimentos_clustering